In [1]:
import torch 
from train import trainAR
from models import *
from matplotlib import pyplot as plt
import build
from tqdm.notebook import tqdm
import pytorch_lightning as pl
import torch.nn.functional as F

In [4]:
import build
loader = build.dataloader(data='dataset_v6', sample_length=4096, part=0.1, batch_size=10, shuffle=True)
print(len(loader))

10


In [5]:
x, y = next(iter(loader))
print(x.shape, y.shape)

torch.Size([10, 256, 4096]) torch.Size([10])


In [6]:
print(x.min(), x.max())

tensor(0.) tensor(1.)


In [7]:
premodel = WaveNet(layers=10, blocks=4, residual_channels=16, skip_channels=16, end_channels=64, classes=256)

In [8]:
class DaNet(pl.LightningModule):
    def __init__(self, model_loader, noise=0.0, lr=0):
        super().__init__()
        self.save_hyperparameters()
        self.model = eval(model_loader)
        self.lr = lr
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, aux = batch
        x += torch.randn_like(x) * self.hparams.noise
        x_hat = self.forward(x)
        loss = F.cross_entropy(x_hat, x)
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [9]:
model = DaNet(module_description(premodel), noise=0.0, lr=0.001)
trainer = pl.Trainer(log_every_n_steps=5)
trainer.fit(model, train_dataloaders=loader)

c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: c:\home\ml\MusicBox\lightning_logs

  | Name  | Type    | Params
----------------------------------
0 | model | WaveNet | 85.8 K
----------------------------------
85.8 K    Trainable params
0         Non-trainable params
85.8 K    Total params
0.343     Total estimated model params size (MB)
c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the

Epoch 181:  90%|█████████ | 9/10 [00:20<00:02,  2.33s/it, loss=3.78, v_num=0] 

c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
model.lr = 1e-3
model.configure_optimizers()
trainer = pl.Trainer(log_every_n_steps=5)
trainer.fit(model, train_dataloaders=loader)

c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 197 K 
-------------------------------------
197 K     Trainable params
0         Non-trainable params
197 K     Total params
0.789     Total estimated model params size (MB)
c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the 

Epoch 4:  40%|████      | 4/10 [00:03<00:04,  1.21it/s, loss=4.16, v_num=4] 

c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
